In [1]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from PIL import Image
import os
from torchvision import transforms

In [3]:
# prompt: count the number of file in one directory

import os

# Get the directory path
directory_path = "E:/Third_year/Machine_Learning_in_Medicine/training_set"

# Count the number of files in the directory
num_files = len(os.listdir(directory_path))

# Print the number of files
print(f"Number of files in the directory: {num_files}")


Number of files in the directory: 1998


In [4]:
X = []
y = []

directory_path = "E:/Third_year/Machine_Learning_in_Medicine/training_set"

for filename in os.listdir(directory_path):
    filepath = os.path.join(directory_path, filename)
    file_content = np.array(Image.open(filepath, 'r'))
    if file_content.shape != (540,800):
        continue
    if "Annotation" in filename:
        y.append(file_content)
    else:
        X.append(file_content)

In [5]:
X_np = np.array(X)
print(X_np.shape)
y_np = np.array(y)
print(y_np.shape)

(975, 540, 800)
(975, 540, 800)


In [6]:
X_train = X_np[:750, :, :]
print(X_train.shape)
X_valid = X_np[750:, :, :]
print(X_valid.shape)
y_train = y_np[:750, :, :]
print(y_train.shape)
y_valid = y_np[750:, :, :]
print(y_valid.shape)


(750, 540, 800)
(225, 540, 800)
(750, 540, 800)
(225, 540, 800)


In [7]:
train_set = []
valid_set = []
for i in range(X_train.shape[0]):
    train_dict = {
        'X': X_train[i,:,:],
        'y': y_train[i,:,:]
    }
    train_set.append(train_dict)

for i in range(X_valid.shape[0]):
    valid_dict = {
        'X': X_valid[i,:,:],
        'y': y_valid[i,:,:]
    }
    valid_set.append(valid_dict)



In [8]:
print(f"Number of training sample: {len(train_set)}")
print(f"Number of validating sample: {len(valid_set)}")

Number of training sample: 750
Number of validating sample: 225


In [9]:
transform_X = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((572,572)),
    transforms.ToTensor(),
])

transform_y = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((388,388)),
    transforms.ToTensor(),
])

In [10]:
class CustomDataset(Dataset):
    def __init__(self, data, transform_X, transform_y):
        self.data = data
        self.transform_X = transform_X
        self.transform_y = transform_y
    def __getitem__(self, index):
        X = self.data[index]['X']
        X = self.transform_X(X)
        y = self.data[index]['y']
        y = self.transform_y(y)
        return X, y
    def __len__(self):
        return len(self.data)


In [21]:
train_cd = CustomDataset(train_set, transform_X, transform_y)
train_loader = DataLoader(train_cd, batch_size = 16, shuffle = True)
valid_cd = CustomDataset(valid_set, transform_X, transform_y)
valid_loader = DataLoader(valid_cd, batch_size = 8, shuffle = False)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [23]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv3x3_1 = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 3),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3),
            nn.ReLU()
        )
        self.conv3x3_2 = nn.Sequential(
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size = 3),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size = 3),
            nn.ReLU()
        )
        self.conv3x3_3 = nn.Sequential(
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3),
            nn.ReLU()
        )
        self.conv3x3_4 = nn.Sequential(
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3),
            nn.ReLU()
        )
        self.conv3x3_5 = nn.Sequential(
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3),
            nn.ReLU()
        )
        self.upconv2x2_1 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=2,stride=2)
        self.conv3x3_6 = nn.Sequential(
            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3),
            nn.ReLU()
        )
        self.upconv2x2_2 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2,stride=2)
        self.conv3x3_7 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3),
            nn.ReLU()
        )
        self.upconv2x2_3 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2,stride=2)
        self.conv3x3_8 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
            nn.ReLU()
        )
        self.upconv2x2_4 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2,stride=2)
        self.conv3x3_9 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
            nn.ReLU()
        )
        self.output = nn.Conv2d(in_channels=64,out_channels=1,kernel_size=1)

    def crop(self, x, out_shape):
        in_shape = x.shape[-1]
        out = x[:,:,(in_shape-out_shape)//2:(in_shape-out_shape)//2+out_shape,(in_shape-out_shape)//2:(in_shape-out_shape)//2+out_shape]
        return(out)
    def forward(self, x):
        cv1 = self.conv3x3_1(x)
        # print(f"Cv1: {cv1.size()}")
        crop_cv1 = self.crop(cv1, 392)

        cv2 = self.conv3x3_2(cv1)
        # print(f"Cv2: {cv2.size()}")
        crop_cv2 = self.crop(cv2, 200)

        cv3 = self.conv3x3_3(cv2)
        # print(f"Cv3: {cv3.size()}")
        crop_cv3 = self.crop(cv3, 104)

        cv4 = self.conv3x3_4(cv3)
        # print(f"Cv4: {cv4.size()}")
        crop_cv4 = self.crop(cv4, 56)

        cv5 = self.conv3x3_5(cv4)
        # print(f"Cv5: {cv5.size()}")

        upcv1 = self.upconv2x2_1(cv5)
        # print(f"UpCv1: {upcv1.size()}         CropCv4: {crop_cv4.size()}")
        
        cv6 = self.conv3x3_6(torch.cat([crop_cv4, upcv1], dim = 1))
        # print(f"Cv6: {cv6.size()}")

        upcv2 = self.upconv2x2_2(cv6)
        # print(f"UpCv2: {upcv2.size()}         CropCv3: {crop_cv3.size()}")
        cv7 = self.conv3x3_7(torch.cat([crop_cv3, upcv2], dim = 1))

        upcv3 = self.upconv2x2_3(cv7)
        # print(f"UpCv3: {upcv3.size()}         CropCv2: {crop_cv2.size()}")
        cv8 = self.conv3x3_8(torch.cat([crop_cv2, upcv3], dim = 1))

        upcv4 = self.upconv2x2_4(cv8)
        # print(f"UpCv4: {upcv4.size()}         CropCv1: {crop_cv1.size()}")
        cv9 = self.conv3x3_9(torch.cat([crop_cv1, upcv4], dim = 1))

        out = self.output(cv9)

        return out


In [24]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    losses = np.array([])
    for X, y in tqdm(dataloader):
        # Compute prediction error
        pred = model(X)
        # class_pred = torch.argmax(pred, dim = -1)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses = np.append(losses, loss.item())

    loss_mean = losses.mean()
    print(f"loss mean per batch: {loss_mean:>7f}")
    return loss_mean

In [25]:
def validating(dataloader, model, loss_fn):

    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X, y
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss per batch: {test_loss:>8f} \n")
    return test_loss

In [26]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer =  torch.optim.RMSprop(model.parameters(),
                          lr=1e-5, weight_decay=1e-8, momentum=0.999, foreach=True)

In [27]:
train_losses = []
val_losses = []
epochs = 4
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_losses.append(train(train_loader, model, criterion, optimizer))
    val_losses.append(validating(valid_loader, model, criterion))
print("Done!")

Epoch 1
-------------------------------


100%|██████████| 47/47 [1:17:04<00:00, 98.39s/it] 


loss mean per batch: 0.000000


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "E:/Third_year/Machine_Learning_in_Medicine")